In [1]:
#Bowel Movement Frequency
#Metabolomics Analysis
#Author: James P. Johnson
#Credit to Christian Diener, PhD as noted
#v 3-7-23

# Load libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns

# Get the library
from arivale_data_interface import *
import arivale_data_interface as adi

# Set display options
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 1000)
from IPython.display import display

# Apply these settings so the library knows where the data is
adi.apply_isb_config()

# If you want plots to show up in the notebook you need to run this
%matplotlib inline
plt.rcParams["figure.figsize"] = [10, 10]

In [8]:
#Obtain Bowel Movement Frequency (BMF) data:
bowel = adi.get_snapshot('assessments')
bowel = bowel.sort_values('days_in_program',ascending=True) # Sort df by days first
bowel = bowel.drop(columns = ['vendor','days_in_program','days_since_first_call','days_since_first_draw']) # Drop days and vendor, unneeded from this point
bowel = bowel.drop_duplicates(subset='public_client_id') # Keep only the first timepoint for each public_client_id
bowel = bowel.fillna(0) # Fill in empty BMF data with zero
bowel = bowel.set_index('public_client_id') # Arrange df by index of PCI
bowel = bowel[['assessment:digestion:bowel-movements:enum']]
bowel = bowel.rename(columns={"assessment:digestion:bowel-movements:enum":"bowel"}) # rename column to "bowel"
bowel = bowel[bowel['bowel'] != 0] #remove participants with no bowel data, reduce from 5764 -> 3955 individuals
bowel_list = ['(1) 2 or fewer times per week', '(2) 3-6 times per week', '(3) 1-3 times daily', '(4) 4+ times daily']
bowel_r = [1,2,3,4] #convert bowel responses to ordinal bowel movement frequency categories
bowel.bowel = bowel.bowel.replace(to_replace = bowel_list, value = bowel_r)
bowel = bowel.reset_index()
# 3955 individuals
bowel

/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_interface/snapshots_interface.py:101: FutureWarning: The default value of regex will change from True to False in a future version.
  t = pandas.to_datetime(x.str.replace("^arivale_snapshot.*(\\d{4}-\\d{2}-\\d{2}_\\d{4})$", "\\1"), format="%Y-%m-%d_%H%M")
/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_interface/snapshots_interface.py:202: FutureWarning: The default value of regex will change from True to False in a future version.
  colnames = hdr.loc[~hdr.str.contains("^#")].head(1).str.replace("\n$","")
/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_interface/snapshots_interface.py:207: FutureWarning: The default value of regex will change from True to False in a future version.
  hdr = hdr.loc[hdr.str.contains("^#")].str.replace("\n$","")
/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_in

,public_client_id,bowel
0,01092042,3
1,01568974,3
2,01684342,2
3,01601230,3
4,01124029,3
...,...,...
3950,01395286,3
3951,01232614,3
3952,01181682,3
3953,01380110,3


In [3]:
#Obtain "eGFR" calculated GFR data (credit: Alexandra Ralevski) code chunk below:
################################################################################################
chems = adi.get_snapshot('chemistries', clean=True)
cl = adi.get_snapshot('clients', clean=True)
chems_dems = join_clients(cl, chems)
#these are the relevant column names
[col for col in chems.columns if 'CREAT' in col]
[col for col in chems.columns if 'GFR' in col]
def f(sex, age, creatinine): 
    if sex == 'F':
        if (creatinine/0.7) <=1:
            return 142*(((creatinine/0.7)**-0.241) * (0.9938**age) * 1.012)
        else:
            return 142*(((creatinine/0.7)**-1.200) * (0.9938**age) * 1.012)
    if sex == 'M':
        if (creatinine/0.9) <=1:
            return 142*(((creatinine/0.9)**-0.302) * (0.9938**age))
        else:
            return 142*(((creatinine/0.7)**-1.200) * (0.9938**age))

creat_calc = chems_dems[['public_client_id','sex', 'age', 'CREATININE_ENZ__SER']]

creat_calc['eGFR'] = creat_calc.apply(lambda x: f(x.sex, x.age, x.CREATININE_ENZ__SER), axis=1)
creat_calc.to_csv('eGFR.csv',index = False)
################################################################################################

#prepare eGFR dataframe:
eGFR = creat_calc
#eGFR = eGFR.set_index('public_client_id')
eGFR = eGFR[['public_client_id','eGFR']] #select relevant columns
#drop duplicate entries to reduce 12316 entries across all individuals
#to 6133 individuals
eGFR = eGFR.dropna()
eGFR = eGFR.drop_duplicates('public_client_id')
eGFR

/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_interface/snapshots_interface.py:101: FutureWarning: The default value of regex will change from True to False in a future version.
  t = pandas.to_datetime(x.str.replace("^arivale_snapshot.*(\\d{4}-\\d{2}-\\d{2}_\\d{4})$", "\\1"), format="%Y-%m-%d_%H%M")
/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_interface/snapshots_interface.py:202: FutureWarning: The default value of regex will change from True to False in a future version.
  colnames = hdr.loc[~hdr.str.contains("^#")].head(1).str.replace("\n$","")
/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_interface/snapshots_interface.py:207: FutureWarning: The default value of regex will change from True to False in a future version.
  hdr = hdr.loc[hdr.str.contains("^#")].str.replace("\n$","")
/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_in

,public_client_id,eGFR
3,01001298,99.024207
4,01001467,100.959522
6,01001548,105.411124
7,01001621,69.439409
9,01001661,120.947576
...,...,...
12307,QIRL,111.687296
12311,ZU273983,118.179626
12312,ZU612255,70.256126
12313,ZU621944,120.472213


In [4]:
# Blood plasma serum metabolites snapshot:
# Get the metabolomics data:
rawmet=adi.get_snapshot('metabolomics_corrected')
rawmet = rawmet.sort_values('days_in_program',ascending=True) # Sort by days
rawmet = rawmet.sort_values('public_client_id',ascending=True) # Sort by PCI
rawmet=rawmet.drop(columns={"sample_id","days_in_program","days_since_first_call","days_since_first_draw","month","weekday","season"}) # Drop unneeded columns
rawmet = rawmet.drop_duplicates('public_client_id') # This line drops duplicates from this snapshot from 3305 participants -> 2043
met_nonnancounts=(~rawmet.isna()).sum() # sum up times any columns have non-NA values
rawmet = rawmet.loc[:, met_nonnancounts/rawmet.shape[0] >= 0.70] #this sum of zero values/num of total rows should less than 30% to proceed with the cohort
rawmet = rawmet.dropna(axis = 0, how = 'all') # Drop patients with all NaN values
rawmet = rawmet.dropna(axis = 1, how = 'all') # Drop patients with all NaN values
rawmet = rawmet.fillna(rawmet.median()) # Fill df NaNs with very small values instead of 0
rawmet
#2043 individuals

/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_interface/snapshots_interface.py:101: FutureWarning: The default value of regex will change from True to False in a future version.
  t = pandas.to_datetime(x.str.replace("^arivale_snapshot.*(\\d{4}-\\d{2}-\\d{2}_\\d{4})$", "\\1"), format="%Y-%m-%d_%H%M")
/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_interface/snapshots_interface.py:202: FutureWarning: The default value of regex will change from True to False in a future version.
  colnames = hdr.loc[~hdr.str.contains("^#")].head(1).str.replace("\n$","")
/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_interface/snapshots_interface.py:207: FutureWarning: The default value of regex will change from True to False in a future version.
  hdr = hdr.loc[hdr.str.contains("^#")].str.replace("\n$","")
/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_in

,public_client_id,35,50,55,62,93,98,111,112,132,136,158,171,179,180,181,182,189,192,194,197,209,212,229,231,234,235,240,241,244,250,252,254,266,267,273,275,278,279,297,302,310,313,330,331,338,339,340,342,344,355,356,358,361,363,376,391,397,407,409,415,424,432,439,444,445,452,460,461,480,482,491,498,501,503,504,512,519,533,535,537,561,563,564,565,566,572,796,798,799,800,803,806,811,815,821,823,825,827,849,...,999946616,999946620,999946623,999946624,999946627,999946632,999946633,999946636,999946639,999946645,999946646,999946657,999946666,999946673,999946674,999946681,999946685,999946690,999946695,999946700,999946701,999946705,999946707,999946710,999946902,999946905,999946911,999946966,999946970,999946972,999946973,999946977,999946986,999946997,999946998,999947006,999947011,999947417,999947437,999947478,999947493,999947642,999947650,999947654,999947670,999947671,999947687,999947708,999947788,999947790,999947802,999947804,999947818,999947820,999947905,999947929,999947955,999947959,999947971,999947977,999947988,999947993,999948001,999948009,999948047,999948076,999948081,999948094,999949463,999949512,999949515,999949517,999949521,999949555,999949557,999949592,999949637,999949679,999949681,999949883,999952025,999952286,999952483,999952501,999952502,999952504,999952524,999952533,999952640,999952661,999952843,999952865,999952867,999952870,999952877,999952909,999953114,999953157,999953267,999954840
0,01000261,0.972222,2.568898,1.155771,0.824435,0.947589,0.772955,0.716831,1.193115,5.372793,24.532961,1.289964,1.135706,0.815813,0.710183,0.986860,0.603223,1.421159,1.128161,1.186826,1.186918,2.183882,1.129249,0.920111,1.374374,1.891255,0.819227,1.204927,1.203517,1.017136,0.921926,1.530622,0.389302,1.364602,1.437235,1.702655,0.954904,2.320103,0.708487,4.268696,2.899063,0.816147,3.386740,1.101887,0.785114,0.943850,0.875629,1.076899,1.014633,1.051367,1.239469,2.139945,1.987275,3.027573,1.165933,1.031538,1.053787,1.165150,1.364448,1.219670,1.115540,0.832081,1.370398,0.920689,1.555217,1.408812,0.800073,1.035181,0.934017,1.245427,1.512853,0.951191,1.512646,4.848959,1.096566,2.289297,1.570808,1.039696,0.889427,0.980195,1.237122,0.948468,1.329892,1.461135,1.029702,1.077272,0.924483,0.496321,1.188124,0.933092,1.196293,0.796418,0.744728,1.375578,0.748462,0.972041,1.600477,1.164032,0.553433,1.373963,...,2.695183,0.819572,0.606842,1.380649,5.777018,1.152754,1.220782,0.167234,0.506707,1.376055,0.585078,0.733665,15.507542,7.548603,0.989995,1.449365,0.960719,1.252370,0.953545,1.799173,2.910742,0.965338,0.651738,0.378580,0.571384,1.854875,2.604398,1.030214,1.039960,1.128947,0.927640,0.924388,1.142827,0.730759,4.020639,1.009418,2.206431,0.885257,1.115327,2.592638,1.835164,0.588192,0.298005,3.765115,1.168743,0.467600,0.918908,2.920539,1.087581,1.021349,0.739217,0.754325,0.462004,0.689655,1.012121,1.000043,0.891436,1.069549,0.671550,0.446172,0.809453,1.266994,4.729395,1.144651,0.790954,1.018050,1.053664,0.997330,0.546438,1.206396,1.436551,1.013768,1.585908,0.483140,0.494334,0.689787,1.214707,1.310367,0.523531,0.570142,0.878512,1.266454,0.934197,1.365642,1.165906,0.659998,0.999463,1.371351,1.089485,1.493849,0.856545,1.678183,1.144330,1.292103,0.715138,1.114859,1.045152,0.592175,0.923405,0.980653
1,01001621,1.819302,0.704097,0.619724,1.014908,1.334809,1.341022,0.971382,0.977802,0.437747,0.597480,0.981717,0.373895,2.085763,0.567837,0.650733,0.983778,1.036290,0.888782,1.192718,0.666124,1.047442,1.219334,0.663285,1.661423,0.694435,2.090090,1.321783,0.546186,0.927013,0.484962,1.264994,0.683409,0.999556,1.063370,0.955280,1.053888,0.834389,0.813427,0.566890,1.378451,1.030161,0.557247,1.359100,1.121047,1.189167,2.339233,1.287490,0.819302,1.526069,0.914017,0.778223,1.437623,0.681799,0.851270,0.929102,1.152256,0.817847,0.871570,1.169466,0.909120,0.644457,1.023283,1.009925,1.015428,0.987975,0.379701,0.962439,1.500350,1.197596,0.760786,0.951191,0.820892,0.538089,0.999284,0.765463,1.128796,0.533141,1.162388,0.894506,1.426137,0.529222,1.009188,1.318555,0.915440,0.804123,0.99

In [5]:
#Get BMI data
bmi = adi.get_snapshot('weight_monthly')
bmi = bmi[['public_client_id','days_in_program','BMI_CALC']] # Keep relevant columns
bmi = bmi.sort_values('days_in_program',ascending=True) # Sort by days in program
bmi = bmi.sort_values('public_client_id',ascending=True) # Sort by PCI
bmi = bmi.drop('days_in_program', axis=1) # Drop days column (unneeded at this point)
bmi = bmi.drop_duplicates('public_client_id') # Drop duplicate PCI
#bmi = bmi.set_index('public_client_id') # Arrange index by PCI
bmi = bmi.dropna() # Drop PCI with no BMI data
bmi #5567 individuals

/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_interface/snapshots_interface.py:101: FutureWarning: The default value of regex will change from True to False in a future version.
  t = pandas.to_datetime(x.str.replace("^arivale_snapshot.*(\\d{4}-\\d{2}-\\d{2}_\\d{4})$", "\\1"), format="%Y-%m-%d_%H%M")
/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_interface/snapshots_interface.py:202: FutureWarning: The default value of regex will change from True to False in a future version.
  colnames = hdr.loc[~hdr.str.contains("^#")].head(1).str.replace("\n$","")
/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_interface/snapshots_interface.py:207: FutureWarning: The default value of regex will change from True to False in a future version.
  hdr = hdr.loc[hdr.str.contains("^#")].str.replace("\n$","")
/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_in

,public_client_id,BMI_CALC
0,01000261,22.594123
3,01000552,31.619378
5,01001181,39.047808
8,01001298,25.948604
14,01001548,36.119222
...,...,...
46475,QIRL,31.942410
46562,ZU273983,32.042139
46566,ZU612255,36.045023
46569,ZU621944,51.577264


In [6]:
#Now, get the Assessments data
bio=adi.get_snapshot('clients')
a = bio['sex']
b = bio['age']
c = bio['public_client_id']
biodf = pd.DataFrame({'sex':a, 'age':b, 'public_client_id':c}) # Get df with desired columns
biodf = biodf[['sex','age','public_client_id']]
#biodf = biodf.set_index('public_client_id') # Arrange index by PCI
biodf = biodf.dropna()
biodf = biodf.replace(to_replace = 'F', value = 'W') # Replace F with W so R Studio doesn't interpret women as a boolean
biodf #6133 individuals

/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_interface/snapshots_interface.py:101: FutureWarning: The default value of regex will change from True to False in a future version.
  t = pandas.to_datetime(x.str.replace("^arivale_snapshot.*(\\d{4}-\\d{2}-\\d{2}_\\d{4})$", "\\1"), format="%Y-%m-%d_%H%M")
/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_interface/snapshots_interface.py:202: FutureWarning: The default value of regex will change from True to False in a future version.
  colnames = hdr.loc[~hdr.str.contains("^#")].head(1).str.replace("\n$","")
/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_interface/snapshots_interface.py:207: FutureWarning: The default value of regex will change from True to False in a future version.
  hdr = hdr.loc[hdr.str.contains("^#")].str.replace("\n$","")
/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_in

,sex,age,public_client_id
0,M,54.0,01000552
1,W,52.0,01000978
2,W,48.0,01001181
3,W,61.0,01001298
4,W,31.0,01001467
...,...,...,...
6128,W,44.0,QIRL
6129,W,32.0,ZU273983
6130,W,35.0,ZU612255
6131,W,37.0,ZU621944


In [9]:
# Merge dfs
output_df = pd.merge(biodf,bmi)
output_df = pd.merge(output_df,eGFR)
output_df = pd.merge(output_df,bowel)
output_df = pd.merge(output_df,rawmet)
metab = output_df

metab = metab.set_index('public_client_id')

#Metadata for R Studio - LIMMA:
metab.iloc[:,:5].to_csv('metab_meta.csv')

#Metabolite count data for R Studio - LIMMA
metab.iloc[:,5:].transpose().to_csv('metab_counts.csv')

#Full dataframe:
metab.to_csv('metab_full.csv')

metab_hm = metab
metab_hm = metab_hm.reset_index() # Remove PCI from the index
metab_hm = metab_hm.drop(columns = ['public_client_id','sex','age','BMI_CALC']) # Drop columns that aren't metabolite or bowel
metab_hm = metab_hm.set_index('bowel') # Arrange index by bowel (BMF)
metab_hm.to_csv('metab_hm.csv')

In [10]:
#Output the metabolomics metatadata df from Arivale
meta = adi.get_snapshot('metabolomics_metadata')
meta.to_csv('metabolomics_fullmetadata.csv')

/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_interface/snapshots_interface.py:101: FutureWarning: The default value of regex will change from True to False in a future version.
  t = pandas.to_datetime(x.str.replace("^arivale_snapshot.*(\\d{4}-\\d{2}-\\d{2}_\\d{4})$", "\\1"), format="%Y-%m-%d_%H%M")
/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_interface/snapshots_interface.py:202: FutureWarning: The default value of regex will change from True to False in a future version.
  colnames = hdr.loc[~hdr.str.contains("^#")].head(1).str.replace("\n$","")
/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_interface/snapshots_interface.py:207: FutureWarning: The default value of regex will change from True to False in a future version.
  hdr = hdr.loc[hdr.str.contains("^#")].str.replace("\n$","")
/users/jajohnso/miniconda3/envs/arivale-py/lib/python3.10/site-packages/arivale_data_in